In [ ]:
import cv2
from pyzbar.pyzbar import decode
import mysql.connector

# Connect to your MySQL database
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="paccopro"
)

cursor = conn.cursor()

# Flag to track processed QR codes in the current session
processed_codes = set()

# Function to check if the scanned QR code exists in the "qr" table
def is_qr_code_present(qcode):
    query = f"SELECT COUNT(*) FROM qr WHERE qcode = '{qcode}'"
    cursor.execute(query)
    result = cursor.fetchone()[0]
    return result > 0

# Function to insert the QR code into the "pp_in" table
def insert_into_in_table(qcode):
    # Get uid corresponding to the qcode from "qr" table
    query_uid = f"SELECT uid FROM qr WHERE qcode = '{qcode}'"
    cursor.execute(query_uid)
    uid = cursor.fetchone()[0]

    # Insert into "pp_in" table with qcode and corresponding uid
    query_insert = f"INSERT INTO pp_in (qcode, uid) VALUES ('{qcode}', {uid})"
    cursor.execute(query_insert)
    conn.commit()

# Main function
def main():
    cap = cv2.VideoCapture(0)

    while True:
        _, frame = cap.read()
        decoded_objects = decode(frame)

        for obj in decoded_objects:
            qcode = obj.data.decode('utf-8')

            # Check if the QR code has already been processed
            if qcode not in processed_codes:
                print(f"Scanned QR Code: {qcode}")

                if is_qr_code_present(qcode):
                    insert_into_in_table(qcode)
                    print("QR code inserted into 'pp_in' table with corresponding uid.")
                else:
                    print("QR code not found in 'qr' table.")

                # Add the processed QR code to the set
                processed_codes.add(qcode)

            # You can add a delay here or use other conditions to control scanning frequency

        cv2.imshow("QR Code Scanner", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    cursor.close()
    conn.close()

if _name_ == "_main_":
    main()

In [ ]:
import cv2
from pyzbar.pyzbar import decode
import mysql.connector

# Connect to your MySQL database
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="paccapro"
)

cursor = conn.cursor()

# Flag to track processed QR codes in the current session
processed_codes = set()

# Function to check if the scanned QR code exists in the "qr" table
def is_qr_code_present(qcode):
    query = f"SELECT COUNT(*) FROM qr WHERE qcode = '{qcode}'"
    cursor.execute(query)
    result = cursor.fetchone()[0]
    return result > 0

# Function to insert the QR code into the "pp_in" table and update flag in "qr" table
def insert_into_in_table(qcode):
    # Get uid corresponding to the qcode from "qr" table
    query_uid = f"SELECT uid FROM qr WHERE qcode = '{qcode}'"
    cursor.execute(query_uid)
    uid = cursor.fetchone()[0]

    # Insert into "pp_in" table with qcode and corresponding uid
    query_insert = f"INSERT INTO pp_in (qcode, uid) VALUES ('{qcode}', {uid})"
    cursor.execute(query_insert)
    conn.commit()

    # Update the flag in "qr" table
    query_update_flag = f"UPDATE qr SET flag = 1 WHERE qcode = '{qcode}'"
    cursor.execute(query_update_flag)
    conn.commit()

# Main function
def main():
    cap = cv2.VideoCapture(1)

    while True:
        _, frame = cap.read()
        decoded_objects = decode(frame)

        for obj in decoded_objects:
            qcode = obj.data.decode('utf-8')

            # Check if the QR code has already been processed
            if qcode not in processed_codes:
                print(f"Scanned QR Code: {qcode}")

                if is_qr_code_present(qcode):
                    insert_into_in_table(qcode)
                    print("QR code inserted into 'pp_in' table with corresponding uid. Flag updated in 'qr' table.")
                else:
                    print("QR code not found in 'qr' table.")

                # Add the processed QR code to the set
                processed_codes.add(qcode)

            # You can add a delay here or use other conditions to control scanning frequency

        cv2.imshow("QR Code Scanner", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    cursor.close()
    conn.close()

if __name__ == "__main__":
    main()
